In [ ]:
import sys
sys.version

In [2]:
import sys
import os

import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import pickle

import random


import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import numpy as np

from  tensorflow.keras.callbacks import ModelCheckpoint
from  collections import Counter

In [26]:
ECG_Scans_processed = pd.read_csv('../Backup/metrics_processed.txt')

ECG_Scans_processed['ScanId'] = ECG_Scans_processed['file_name'].apply(lambda x:x.split('_')[0])
categorical_predictors = ['QRS_Axis']
ECG_Scans_processed_modified = pd.get_dummies(ECG_Scans_processed, columns= categorical_predictors)

predictors = list(ECG_Scans_processed_modified.columns)
for eEle in ['ScanId', 'file_name']:
    predictors.remove(eEle)
print(predictors)


PredictorsDF = ECG_Scans_processed_modified.loc[:, predictors]
min_max_scaler = preprocessing.MinMaxScaler()
PredictorsDF   = pd.DataFrame( min_max_scaler.fit_transform(PredictorsDF) )
ECG_Scans_processed_modified = ECG_Scans_processed_modified.loc[ :, [ 'ScanId', 'file_name'] ].join(PredictorsDF)

ScanDictLeadII = {}
for escan in ECG_Scans_processed_modified.values.tolist():
    scandId = escan[0]
    if scandId in ScanDictLeadII:
        ScanDictLeadII[scandId].append(escan[2:])
    else:
        ScanDictLeadII[scandId] = []
        ScanDictLeadII[scandId].append(escan[2:])      
for scanId in ScanDictLeadII:
    ScanDictLeadII[scanId].sort( key=lambda x: x[0] ) # In place sorting



['heart_rate', 'P_Wave', 'PR_Segment', 'PR_Interval', 'QRS_Complex', 'ST_Segment', 'T_Wave', 'QT_Interval', 'QTc_Interval', 'RR_Interval', 'QRS_Axis_EAD', 'QRS_Axis_LAD', 'QRS_Axis_Normal', 'QRS_Axis_RAD']


In [38]:
baseDir = '../Backup/raw_ecg_processed_lead_new/'


ScanDictRawECG = {}
for scanIdx, scanId in enumerate( os.listdir(baseDir)):
    if scanId not in ScanDictRawECG:
        ScanDictRawECG[scanId] = []
    tlist = open(os.path.join(baseDir, scanId) ).read().split('\n')
    for eentry in tlist[250*5 : -250*5]:        
        eentry = eentry.split('\t')
        if len(eentry) >= 12:
            eentry = [ int(ee) for ee in eentry[0:12] ]
            ScanDictRawECG[scanId].append(eentry)        
            

In [43]:
resDir='../Backup/classifier_data_set.txt'
# Loading Scands Ids with (AR)Rhythm Type

list_of_ecg_scans_with_rhythm_type = {}
for erecord in open(resDir).readlines():
    erecord = erecord.strip('\n,').split(',')[0:3] # Col1: ScanId, Col2: Normal/Abnormal Col3: Specific Type of DysRhythm (if any)
    list_of_ecg_scans_with_rhythm_type[erecord[0]] = erecord

print(list_of_ecg_scans_with_rhythm_type['CLI0200@5@73201718243468764'])

['CLI0200@5@73201718243468764', 'NORMAL', 'Normal ECG']


In [49]:
noData = 0 
RawEcgScan    = []
LeadIIEcgScan = []
scanTypesInt  = []
for eScanId in set( ScanDictLeadII.keys()  ).intersection ( set( ScanDictRawECG.keys() ) ) :
    if len(ScanDictRawECG[eScanId]) == 0:
        noData += 1
    RawEcgScan.append    ( ScanDictRawECG[eScanId] )
    LeadIIEcgScan.append ( ScanDictLeadII[eScanId] )
    if list_of_ecg_scans_with_rhythm_type[eScanId][1] == 'NORMAL':
        scanTypesInt.append(0)
    else:
        scanTypesInt.append(1)
        

In [58]:
Counter(scanTypesInt)

Counter({1: 330, 0: 764})